In [13]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from newspaper import Article
import nltk


def fetch_article_summary(link):
    """
    Fetch and summarize the article content from a URL.
    """
    try:
        # Use Newspaper3k with headers
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Newspaper3k failed for {link}: {str(e)}. Falling back to BeautifulSoup.")

        # Fallback to BeautifulSoup
        try:
            response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join([p.get_text() for p in paragraphs])
            return content[:500] + "..." if len(content) > 500 else content
        except Exception as bs_error:
            print(f"[ERROR] BeautifulSoup also failed for {link}: {str(bs_error)}")
            return "No summary available."




def get_market_news(tickers):
    """
    Fetch market news for the current day, capturing all available fields and generating summaries.
    Only processes news items with 'type' set to 'story'.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)

        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Filter news to include only today's and yesterday's articles
                    if publish_date >= one_day_ago:
                        # Only summarize articles with type 'story'
                        if item.get('type', '').lower() == 'story':
                            link = item.get('link', '')
                            summary = fetch_article_summary(link) if link else "No summary available."

                            news_item = {
                                'ticker': ticker,
                                'title': item.get('title', ''),
                                'publisher': item.get('publisher', ''),
                                'link': link,
                                'publish_date': datetime.fromtimestamp(publish_timestamp),
                                'summary': summary,  # Include the generated summary
                                'type': item.get('type', ''),  # Original type from Yahoo API
                                'related_tickers': ', '.join(item.get('relatedTickers', [])),  # Comma-separated related tickers
                            }
                            all_news.append(news_item)
                        else:
                            print(f"[INFO] Skipping non-story type: {item.get('type', '')}")
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")

        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")

    print(f"[INFO] Fetched {len(all_news)} news articles.")
    return pd.DataFrame(all_news)


def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file locally.
    """
    try:
        if df.empty:
            print("[INFO] No news data to save.")
            return None

        os.makedirs(output_dir, exist_ok=True)
        filename = f"market_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False, encoding="utf-8")
        print(f"[INFO] News data saved locally at: {filepath}")
        return filepath
    except Exception as e:
        print(f"[ERROR] Failed to save CSV: {str(e)}")
        return None


def main():
    """
    Main function for fetching and saving market news locally.
    """
    try:
        # Fetch general market news
        indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
        market_news = get_market_news(tickers=indices)
        if not market_news.empty:
            market_news['category'] = 'General'  # Add category for general market

        # Fetch tech stock news
        tech_stocks = ['AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR', 'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG', 'AVGO', 'SMCI','GLW', 'HAL', 'LMT', 'AMZ', 'CRM', 'NOW', 'CHTR', 'TDS', 'META']
        tech_news = get_market_news(tickers=tech_stocks)
        if not tech_news.empty:
            tech_news['category'] = 'Tech'  # Add category for tech stocks

        # Combine news
        combined_news = pd.concat([market_news, tech_news], ignore_index=True)

        # Save to CSV locally
        if not combined_news.empty:
            save_to_csv(combined_news)
        else:
            print("[INFO] No news data to save.")
    except Exception as e:
        print(f"[ERROR] Error in main function: {e}")


if __name__ == "__main__":
    main()


[INFO] Skipping non-story type: VIDEO
[INFO] Skipping non-story type: VIDEO
[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/equities-mostly-rise-markets-analyze-214815922.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
***************************

ASTS: Failed to retrieve the news and received faulty response instead.


[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/3-stocks-watch-satellite-communication-153900448.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
. Falling back to BeautifulSou

GLW: Failed to retrieve the news and received faulty response instead.
HAL: Failed to retrieve the news and received faulty response instead.
LMT: Failed to retrieve the news and received faulty response instead.
AMZ: Failed to retrieve the news and received faulty response instead.
CRM: Failed to retrieve the news and received faulty response instead.
NOW: Failed to retrieve the news and received faulty response instead.
CHTR: Failed to retrieve the news and received faulty response instead.
TDS: Failed to retrieve the news and received faulty response instead.
META: Failed to retrieve the news and received faulty response instead.


[INFO] Fetched 68 news articles.
[INFO] News data saved locally at: market_news\market_news_20241122_085340.csv


In [20]:
from textblob import TextBlob  # Using TextBlob for sentiment analysis

def calculate_sentiment(text):
    """
    Calculate the sentiment score of a text using TextBlob.
    Returns a polarity score between -1 and 1.
    """
    try:
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return 0  # Default to neutral if analysis fails


def label_sentiment(score):
    """
    Label the sentiment based on the score.
    """
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"


def get_market_news(tickers):
    """
    Fetch market news for the current day, capturing all available fields and generating summaries.
    Only processes news items with 'type' set to 'story'.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)

        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    # Filter news to include only today's and yesterday's articles
                    if publish_date >= one_day_ago:
                        # Only summarize articles with type 'story'
                        if item.get('type', '').lower() == 'story':
                            link = item.get('link', '')
                            summary = fetch_article_summary(link) if link else "No summary available."

                            # Sentiment Analysis
                            sentiment_score = calculate_sentiment(summary)
                            sentiment_label = label_sentiment(sentiment_score)

                            news_item = {
                                'ticker': ticker,
                                'title': item.get('title', ''),
                                'summary': summary,
                                'publisher': item.get('publisher', ''),
                                'link': link,
                                'publish_date': datetime.fromtimestamp(publish_timestamp),
                                'type': item.get('type', ''),
                                'related_tickers': ', '.join(item.get('relatedTickers', [])),
                                'source': 'yahoo',
                                'overall_sentiment_score': sentiment_score,
                                'overall_sentiment_label': sentiment_label,
                            }
                            all_news.append(news_item)
                        else:
                            print(f"[INFO] Skipping non-story type: {item.get('type', '')}")
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")

        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")

    print(f"[INFO] Fetched {len(all_news)} news articles.")
    return pd.DataFrame(all_news)


def save_to_csv(df, output_dir="market_news"):
    """
    Save processed news data to a CSV file locally.
    """
    try:
        if df.empty:
            print("[INFO] No news data to save.")
            return None

        # Reorder columns
        column_order = ['ticker', 'title', 'summary', 'publisher', 'link', 'publish_date', 'type', 'related_tickers', 'source', 'overall_sentiment_score', 'overall_sentiment_label']
        df = df[column_order]

        os.makedirs(output_dir, exist_ok=True)
        filename = f"market_news_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False, encoding="utf-8")
        print(f"[INFO] News data saved locally at: {filepath}")
        return filepath
    except Exception as e:
        print(f"[ERROR] Failed to save CSV: {str(e)}")
        return None


def main():
    """
    Main function for fetching and saving market news locally.
    """
    try:
        # Fetch general market news
        indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
        market_news = get_market_news(tickers=indices)
        if not market_news.empty:
            market_news['category'] = 'General'  # Add category for general market

        # Fetch tech stock news
        tech_stocks = ['AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR', 'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG', 'AVGO', 'SMCI','GLW', 'HAL', 'LMT', 'AMZ', 'CRM', 'NOW', 'CHTR', 'TDS', 'META']
        tech_news = get_market_news(tickers=tech_stocks)
        if not tech_news.empty:
            tech_news['category'] = 'Tech'  # Add category for tech stocks

        # Combine news
        combined_news = pd.concat([market_news, tech_news], ignore_index=True)

        # Save to CSV locally
        if not combined_news.empty:
            save_to_csv(combined_news)
        else:
            print("[INFO] No news data to save.")
    except Exception as e:
        print(f"[ERROR] Error in main function: {e}")


if __name__ == "__main__":
    main()


[INFO] Skipping non-story type: VIDEO
[INFO] Skipping non-story type: VIDEO
[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/equities-mostly-rise-markets-analyze-214815922.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
***************************

ASTS: Failed to retrieve the news and received faulty response instead.


[ERROR] Newspaper3k failed for https://finance.yahoo.com/news/3-stocks-watch-satellite-communication-153900448.html: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
. Falling back to BeautifulSou

GLW: Failed to retrieve the news and received faulty response instead.
HAL: Failed to retrieve the news and received faulty response instead.
LMT: Failed to retrieve the news and received faulty response instead.
AMZ: Failed to retrieve the news and received faulty response instead.
CRM: Failed to retrieve the news and received faulty response instead.
NOW: Failed to retrieve the news and received faulty response instead.
CHTR: Failed to retrieve the news and received faulty response instead.
TDS: Failed to retrieve the news and received faulty response instead.
META: Failed to retrieve the news and received faulty response instead.


[INFO] Fetched 68 news articles.
[INFO] News data saved locally at: market_news\market_news_20241122_125119.csv


In [ ]:
# Current works as of 11/22 12:37 in clound but has issues.
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import logging
from google.cloud import bigquery

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(tickers):
    """
    Fetch market news for specified tickers using Yahoo Finance API.
    """
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        try:
            logger.info(f"Fetching news for ticker: {ticker}")
            stock = yf.Ticker(ticker)
            news = stock.news
            for item in news:
                try:
                    publish_date = datetime.fromtimestamp(item.get('providerPublishTime', 0)).date()
                    if publish_date >= one_day_ago:
                        all_news.append({
                            'ticker': ticker,
                            'title': item.get('title', ''),
                            'summary': item.get('summary', ''),
                            'publisher': item.get('publisher', ''),
                            'link': item.get('link', ''),
                            'publish_date': item.get('providerPublishTime', 0),
                            'type': item.get('type', ''),
                            'related_tickers': ', '.join(item.get('relatedTickers', [])),
                            'source': 'yahoo',
                            'overall_sentiment_score': None,  # Placeholder
                            'overall_sentiment_label': None  # Placeholder
                        })
                except Exception as e:
                    logger.error(f"Error processing news item for {ticker}: {str(e)}")
        except Exception as e:
            logger.error(f"Error fetching news for {ticker}: {str(e)}")
    
    logger.info(f"Total news items fetched: {len(all_news)}")
    return pd.DataFrame(all_news)

def save_to_bigquery(df, project_id, dataset_id, table_id):
    """
    Save processed news data to Google BigQuery.
    """
    try:
        if df.empty:
            logger.warning("No data to save.")
            return None
        
        client = bigquery.Client()
        table_ref = f"{project_id}.{dataset_id}.{table_id}"
        job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition="WRITE_APPEND")

        logger.info(f"Saving data to BigQuery table: {table_ref}")
        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()  # Wait for job completion
        logger.info("Data successfully saved to BigQuery.")
        return table_ref
    except Exception as e:
        logger.error(f"Error saving data to BigQuery: {str(e)}")
        return None

def main(request):
    """
    Cloud Function entry point.
    """
    # Configuration
    project_id = "trendsense"
    dataset_id = "market_data"
    table_id = "Yahoo_News_Extract"
    tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']

    try:
        # Fetch and process market news
        news_df = get_market_news(tickers)
        if news_df.empty:
            logger.warning("No news data retrieved.")
            return "No news data retrieved.", 200

        # Save to BigQuery
        save_result = save_to_bigquery(news_df, project_id, dataset_id, table_id)
        if save_result:
            return f"Data successfully saved to BigQuery: {save_result}", 200
        else:
            return "Failed to save data to BigQuery.", 500
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")
        return f"Internal Server Error: {str(e)}", 500


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import logging
import os
from google.cloud import bigquery

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(api_key):
    """
    Retrieve market news from Alpha Vantage API.
    """
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'apikey': api_key
    }
    
    try:
        logger.info("Fetching market news")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        news_data = response.json()

        if 'Note' in news_data:
            logger.warning(f"API limit message: {news_data['Note']}")
            return []

        if 'feed' in news_data:
            # Filter articles to include only today's articles
            today = datetime.now().strftime('%Y-%m-%d')
            filtered_news = [
                item for item in news_data['feed'] 
                if datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S").strftime('%Y-%m-%d') == today
            ]
            logger.info(f"Retrieved {len(filtered_news)} news items for today.")
            return filtered_news
        else:
            logger.warning("No news found in response")
            return []
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {str(e)}")
        return []
    except Exception as e:
        logger.error(f"Unexpected error occurred: {str(e)}")
        return []

def process_news_items(news_items):
    """
    Process news items into a structured DataFrame, including sentiment.
    """
    try:
        processed_items = []
        for item in news_items:
            # Convert publish_date format
            raw_date = item.get('time_published', '')
            try:
                formatted_date = datetime.strptime(raw_date, "%Y%m%dT%H%M%S").strftime("%m/%d/%Y %H:%M")
            except ValueError:
                formatted_date = "Invalid Date"

            processed_item = {
                'ticker': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'title': item.get('title', ''),
                'summary': item.get('summary', ''),
                'publisher': item.get('source', ''),
                'link': item.get('url', ''),
                'publish_date': formatted_date,
                'related_tickers': ', '.join([ts['ticker'] for ts in item.get('ticker_sentiment', [])]),
                'source': 'Alpha',
                'overall_sentiment_score': item.get('overall_sentiment_score', ''),
                'overall_sentiment_label': item.get('overall_sentiment_label', '')
            }
            processed_items.append(processed_item)
        
        # Reorder columns
        column_order = ['ticker', 'title', 'summary', 'publisher', 'link', 'publish_date', 
                        'related_tickers', 'source', 'overall_sentiment_score', 'overall_sentiment_label']
        return pd.DataFrame(processed_items)[column_order]
    except Exception as e:
        logger.error(f"Error processing news items: {str(e)}")
        return pd.DataFrame()

def save_to_bigquery(df, project_id, dataset_id, table_id):
    """
    Save processed news data to Google BigQuery using auto-detect schema.

    Parameters:
    df (pd.DataFrame): DataFrame containing news data
    project_id (str): Google Cloud Project ID
    dataset_id (str): BigQuery dataset ID
    table_id (str): BigQuery table ID
    """
    try:
        if df.empty:
            logger.warning("No news data to save.")
            return None

        # Initialize BigQuery client
        client = bigquery.Client()

        # Define BigQuery table ID
        table_ref = f"{project_id}.{dataset_id}.{table_id}"

        # Load data into BigQuery with auto-detect schema
        job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition="WRITE_APPEND")
        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()  # Wait for the job to complete
        logger.info(f"Data successfully saved to BigQuery: {table_ref}")
        return table_ref
    except Exception as e:
        logger.error(f"Error saving data to BigQuery: {str(e)}")
        return None

def main(request):
    """
    Google Cloud Function handler for fetching, processing, and saving market news data.
    """
    # Configuration
    api_key = "FLGDYAANWX6EFL9P"  # Alpha Vantage API key from environment variable
    project_id = "trendsense"                     # Your Google Cloud project ID
    dataset_id = "market_data"                    # BigQuery dataset ID
    table_id = "News_Alpha_Extract"               # BigQuery table ID

    try:
        # Fetch market news
        news_items = get_market_news(api_key)
        if not news_items:
            logger.warning("No news data retrieved.")
            return "No news data retrieved.", 200

        # Process news items
        news_df = process_news_items(news_items)

        # Save processed news data to BigQuery
        save_result = save_to_bigquery(news_df, project_id, dataset_id, table_id)

        if save_result:
            return f"News data saved successfully to BigQuery table: {save_result}", 200
        else:
            return "Failed to save news data to BigQuery.", 500
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")
        return f"Internal Server Error: {str(e)}", 500
        function: {str(e)}")
            return f"Internal Server Error: {str(e)}", 500

In [ ]:
# Current works as of 11/22 12:37 in clound but has issues.
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import logging
from google.cloud import bigquery
from textblob import TextBlob


# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def label_sentiment(score):
    """
    Label the sentiment based on the score.
    """
    if score is None:
        return "Unknown"  # Handle cases where score is None
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"

def calculate_sentiment(text):
    try:
        if not text:
            print("[WARNING] Empty text provided for sentiment analysis.")
            return None
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return None  # Return None for failed analysis

def get_market_news(tickers):
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            news = stock.news
            for item in news:
                try:
                    title = item.get('title', '')
                    print(f"[DEBUG] Processing title: {title}")

                    sentiment_score = calculate_sentiment(title)
                    sentiment_label = label_sentiment(sentiment_score)

                    news_item = {
                        'ticker': ticker,
                        'title': title,
                        'publisher': item.get('publisher', ''),
                        'link': item.get('link', ''),
                        'publish_date': datetime.fromtimestamp(item.get('providerPublishTime', 0)).isoformat(),
                        'type': item.get('type', ''),
                        'related_tickers': ', '.join(item.get('relatedTickers', [])),
                        'source': 'yahoo',
                        'overall_sentiment_score': sentiment_score,
                        'overall_sentiment_label': sentiment_label,
                    }
                    all_news.append(news_item)
                except Exception as e:
                    print(f"[ERROR] Error processing news item: {e}")
        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {e}")

    return pd.DataFrame(all_news)


def save_to_bigquery(df, project_id, dataset_id, table_id):
    """
    Save processed news data to Google BigQuery.
    """
    try:
        if df.empty:
            logger.warning("No data to save.")
            return None
        
        client = bigquery.Client()
        table_ref = f"{project_id}.{dataset_id}.{table_id}"
        job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition="WRITE_TRUNCATE")

        logger.info(f"Saving data to BigQuery table: {table_ref}")
        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()  # Wait for job completion
        logger.info("Data successfully saved to BigQuery.")
        return table_ref
    except Exception as e:
        logger.error(f"Error saving data to BigQuery: {str(e)}")
        return None

def fetch_market_news(request):
    """
    Cloud Function entry point.
    """
    # Configuration
    project_id = "trendsense"
    dataset_id = "market_data"
    table_id = "News_Yahoo_Extract"
    tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']

    try:
        # Fetch and process market news
        news_df = get_market_news(tickers)
        if news_df.empty:
            logger.warning("No news data retrieved.")
            return "No news data retrieved.", 200

        # Save to BigQuery
        save_result = save_to_bigquery(news_df, project_id, dataset_id, table_id)
        if save_result:
            return f"Data successfully saved to BigQuery: {save_result}", 200
        else:
            return "Failed to save data to BigQuery.", 500
    except Exception as e:
        logger.error(f"Error in main function: {str(e)}")
        return f"Internal Server Error: {str(e)}", 500
